In [1]:
include("ecow.jl")

ECow

In [71]:
using Plots
const mm = Plots.mm
fntsm = Plots.font("times", 12)
fntlg = Plots.font("times", 14)
default(
    titlefont=fntlg,
    guidefont=fntlg,
    tickfont=fntsm,
    legendfont=fntsm,
    left_margin=3mm,
    bottom_margin=10mm,
    top_margin=0mm,
    right_margin=0mm,
    size=(500,300),
    xlabel="Days of the Season\n",
    label="",
    w=3,
    c="#00467F"
)
gr()

Plots.GRBackend()

In [77]:
cow = ECow.Cow()
herbage = ECow.FoodOffer(ECow.Pasture(), 35.0)
supplement = ECow.FoodOffer(ECow.PKE(), 2.0)

milkingplan(t,n=365) = vcat(trues(t), falses(n-t))

results = Dict(
    :bcs         => vcat(3.2, zeros(365)),
    :maintenance => vcat(63, zeros(365)),
    :lactation   => milkingplan(40 * 7),  # milk for 40 weeks
    :supplement  => zeros(365),
    :herbage     => zeros(365),
    :milksolids  => zeros(365)
)

for day in 1:365
    (new_bcs, new_maintenance, actual_herbage_intake, actual_supplement_intake, milksolids) = ECow.updateday!(
        cow, day, results[:bcs][day], results[:maintenance][day], results[:lactation][day], supplement, herbage
    )
    results[:bcs][day+1]         = new_bcs
    results[:maintenance][day+1] = new_maintenance
    results[:supplement][day]    = actual_supplement_intake
    results[:herbage][day]       = actual_herbage_intake
    results[:milksolids][day]    = milksolids
end

println("Total Milk Production: $(sum(results[:milksolids]))")
println("Total Herbage Consumption: $(sum(results[:herbage]))")
println("Total PKE Consumption: $(sum(results[:supplement]))")

Total Milk Production: 323.3706873358398
Total Herbage Consumption: 4154.016005819228
Total PKE Consumption: 730.0


In [78]:
bcs_plot = plot(results[:bcs],
    ylabel="BCS (Units)",
    title="(a)",
    ylims=(0,5)
)

mai_plot = plot(results[:maintenance],
    ylabel="Maintenance (MJ/Day)",
    title="(b)",
    ylims=(50,80)
)

her_plot = plot(
    ylabel="Feed Intake (kg/Day)",
    title="(c)",
    ylims=(0,14),
    legend=:topleft
)
plot!(results[:herbage], label="Herbage")
plot!(results[:supplement], label="Supplement", linestyle=:dash)

mlk_plot = plot(results[:milksolids],
    ylabel="Milk Solids (kg/Day)",
    title="(d)",
    ylims=(0,1.5)
)

plot(bcs_plot, mai_plot, her_plot, mlk_plot, size=(1000, 600))

savefig("moo_example.pdf")